In [23]:
import pandas as pd 
import numpy as np
import seaborn as sns
import category_encoders as ce

import matplotlib.pyplot as plt 
%matplotlib inline


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, log_loss, matthews_corrcoef, balanced_accuracy_score
import xgboost as xgb
from imblearn.over_sampling import SMOTE


In [24]:
# Đọc file CSV
df = pd.read_csv('Traffic_Crashes_-_Crashes.csv')

In [25]:
df.describe()

,POSTED_SPEED_LIMIT,LANE_CNT,STREET_NO,BEAT_OF_OCCURRENCE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
count,794956.000000,1.990060e+05,794956.000000,794951.000000,794956.000000,793215.000000,793215.000000,793215.000000,793215.000000,793215.000000,793215.000000,793215.0,794956.000000,794956.000000,794956.000000,789567.000000,789567.000000
mean,28.402547,1.333032e+01,3688.085435,1242.916397,2.034919,0.189448,0.001195,0.020032,0.107025,0.061195,2.004313,0.0,13.202800,4.122344,6.787978,41.854788,-87.673450
std,6.179272,2.961623e+03,2887.990566,705.268208,0.452475,0.565654,0.037456,0.165673,0.422095,0.318451,1.158715,0.0,5.568727,1.980411,3.405398,0.337523,0.686153
min,0.000000,0.000000e+00,0.000000,111.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,-87.936193
25%,30.000000,2.000000e+00,1248.000000,714.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,9.000000,2.000000,4.000000,41.782429,-87.721709
50%,30.000000,2.000000e+00,3201.000000,1211.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,14.000000,4.000000,7.000000,41.874743,-87.674094
75%,30.000000,4.000000e+00,5600.000000,1822.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,17.000000,6.000000,10.000000,41.924336,-87.633300
max,99.000000,1.191625e+06,451100.000000,6100.000000,18.000000,21.000000,4.000000,10.000000,21.000000,15.000000,61.000000,0.0,23.000000,7.000000,12.000000,42.022780,0.000000


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 794956 entries, 0 to 794955
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                794956 non-null  object 
 1   CRASH_DATE_EST_I               59606 non-null   object 
 2   CRASH_DATE                     794956 non-null  object 
 3   POSTED_SPEED_LIMIT             794956 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         794956 non-null  object 
 5   DEVICE_CONDITION               794956 non-null  object 
 6   WEATHER_CONDITION              794956 non-null  object 
 7   LIGHTING_CONDITION             794956 non-null  object 
 8   FIRST_CRASH_TYPE               794956 non-null  object 
 9   TRAFFICWAY_TYPE                794956 non-null  object 
 10  LANE_CNT                       199006 non-null  float64
 11  ALIGNMENT                      794956 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [27]:
print('This data has {} rows and {} columns'.format(df.shape[0], df.shape[1]))

This data has 794956 rows and 48 columns


In [28]:
print("Dataset missing values: \n", df.isna().sum())
plt.show()

Dataset missing values: 
 CRASH_RECORD_ID                       0
CRASH_DATE_EST_I                 735350
CRASH_DATE                            0
POSTED_SPEED_LIMIT                    0
TRAFFIC_CONTROL_DEVICE                0
DEVICE_CONDITION                      0
WEATHER_CONDITION                     0
LIGHTING_CONDITION                    0
FIRST_CRASH_TYPE                      0
TRAFFICWAY_TYPE                       0
LANE_CNT                         595950
ALIGNMENT                             0
ROADWAY_SURFACE_COND                  0
ROAD_DEFECT                           0
REPORT_TYPE                       23258
CRASH_TYPE                            0
INTERSECTION_RELATED_I           612741
NOT_RIGHT_OF_WAY_I               758229
HIT_AND_RUN_I                    546330
DAMAGE                                0
DATE_POLICE_NOTIFIED                  0
PRIM_CONTRIBUTORY_CAUSE               0
SEC_CONTRIBUTORY_CAUSE                0
STREET_NO                             0
STREET_DIRECTI

In [29]:
# Danh sách các cột cần xóa
columns_to_drop = [
    "CRASH_RECORD_ID", "CRASH_DATE_EST_I","CRASH_DATE", "LANE_CNT", "REPORT_TYPE",
    "INTERSECTION_RELATED_I", "NOT_RIGHT_OF_WAY_I", "HIT_AND_RUN_I",
    "DATE_POLICE_NOTIFIED", "SEC_CONTRIBUTORY_CAUSE", "STREET_NO",
    "STREET_DIRECTION", "STREET_NAME", "BEAT_OF_OCCURRENCE",
    "PHOTOS_TAKEN_I", "STATEMENTS_TAKEN_I", "DOORING_I", "WORK_ZONE_I",
    "WORK_ZONE_TYPE", "WORKERS_PRESENT_I", "INJURIES_TOTAL", "INJURIES_FATAL",
    "INJURIES_INCAPACITATING", "INJURIES_NON_INCAPACITATING",
    "INJURIES_REPORTED_NOT_EVIDENT", "INJURIES_NO_INDICATION", "INJURIES_UNKNOWN", "LATITUDE", "LONGITUDE", "LOCATION", "MOST_SEVERE_INJURY"
]

# Xóa các cột, bỏ qua các cột không tồn tại
df = df.drop(columns=columns_to_drop, errors='ignore')

# Kiểm tra kết quả
print("Danh sách cột sau khi xóa:")
print(df.columns)


Danh sách cột sau khi xóa:
Index(['POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'CRASH_TYPE', 'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE', 'NUM_UNITS',
       'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH'],
      dtype='object')


In [30]:
unique_count = df['CRASH_TYPE'].nunique()
print(f"Số lượng giá trị khác nhau trong CRASH_TYPE: {unique_count}")

value_counts = df['CRASH_TYPE'].value_counts()
print("\nSố lần xuất hiện của từng giá trị trong CRASH_TYPE:")
print(value_counts)


Số lượng giá trị khác nhau trong CRASH_TYPE: 2

Số lần xuất hiện của từng giá trị trong CRASH_TYPE:
CRASH_TYPE
NO INJURY / DRIVE AWAY              583177
INJURY AND / OR TOW DUE TO CRASH    211779
Name: count, dtype: int64


In [31]:

# Loại bỏ các hàng null
df = df.dropna()

In [32]:



# Tạo cột nhãn nhị phân 
df['CRASH_TYPE'] = df['CRASH_TYPE'].apply(
    lambda x: 1 if x in ['INJURY AND / OR TOW DUE TO CRASH'] else 0
)

# In số lượng mẫu theo nhãn
print("Số lượng mẫu theo nhãn:")
print(df['CRASH_TYPE'].value_counts())

# Tách dữ liệu và nhãn
X = df.drop(columns=['CRASH_TYPE'])
y = df['CRASH_TYPE']

# Mã hóa các cột phân loại (Label Encoding)
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))
    label_encoders[column] = le

# Xử lý giá trị thiếu
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Áp dụng SMOTE chỉ cho tập huấn luyện (train set)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Kiểm tra số lượng mẫu sau khi áp dụng SMOTE
print(f"Số lượng mẫu sau khi áp dụng SMOTE - lớp 0: {sum(y_train_res == 0)}, lớp 1: {sum(y_train_res == 1)}")

# Tính toán scale_pos_weight
pos_weight = (len(y_train_res) - sum(y_train_res)) / sum(y_train_res)
print(f"Scale_pos_weight: {pos_weight:.2f}")

# Huấn luyện mô hình XGBoost
model_xgb = xgb.XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    scale_pos_weight=pos_weight  # Thêm trọng số cho lớp 1
)

# Tuning tham số với GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

grid_search = GridSearchCV(estimator=model_xgb, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_res, y_train_res)

# In ra tham số tối ưu
print(f"Best Parameters: {grid_search.best_params_}")

# Dự đoán và đánh giá
best_model = grid_search.best_estimator_
y_pred_xgb = best_model.predict(X_test)
y_prob_xgb = best_model.predict_proba(X_test)[:, 1]  # Dự đoán xác suất lớp 1

# Báo cáo kết quả
print("Classification Report (XGBoost):")
print(classification_report(y_test, y_pred_xgb))
print(f"Accuracy (XGBoost): {accuracy_score(y_test, y_pred_xgb):.2f}")

# AUC Score
auc_score = roc_auc_score(y_test, y_prob_xgb)
print(f"AUC (XGBoost): {auc_score:.2f}")

# Log Loss
log_loss_value = log_loss(y_test, y_prob_xgb)
print(f"Log Loss: {log_loss_value:.2f}")

# Matthews Correlation Coefficient (MCC)
mcc_value = matthews_corrcoef(y_test, y_pred_xgb)
print(f"Matthews Correlation Coefficient (MCC): {mcc_value:.2f}")

# Balanced Accuracy
balanced_accuracy_value = balanced_accuracy_score(y_test, y_pred_xgb)
print(f"Balanced Accuracy: {balanced_accuracy_value:.2f}")



Số lượng mẫu theo nhãn:
CRASH_TYPE
0    583177
1    211779
Name: count, dtype: int64
Số lượng mẫu sau khi áp dụng SMOTE - lớp 0: 408251, lớp 1: 408251
Scale_pos_weight: 1.00
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best Parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'subsample': 0.8}
Classification Report (XGBoost):
              precision    recall  f1-score   support

           0       0.85      0.91      0.88    174926
           1       0.70      0.54      0.61     63561

    accuracy                           0.81    238487
   macro avg       0.77      0.73      0.74    238487
weighted avg       0.81      0.81      0.81    238487

Accuracy (XGBoost): 0.81
AUC (XGBoost): 0.84
Log Loss: 0.42
Matthews Correlation Coefficient (MCC): 0.50
Balanced Accuracy: 0.73
